In [15]:
import tensorflow as tf

In [20]:
class LinearRegression(tf.keras.Model):
  def __init__(self):
    super(LinearRegression, self).__init__()
    self.linear_layer = tf.keras.layers.Dense(1, activation=None)

  def call(self, x):
    y_pred = self.linear_layer(x)

    return y_pred

In [21]:
@tf.function
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.squire(y_pred - y))

optimizer = tf.optimizers.SGD(0.01)

summary_writer = tf.summary.create_file_writer('./tensorboard_log')

In [22]:
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = mse_loss(y_pred, y)

  with summary_writer.as_default():
    tf.summary.scalar('loss', loss, step=optimizer.iterations)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [26]:
x_train = [1.0, 2.0, 3.0, 4.0]
y_train = [2.0, 4.0, 6.0, 8.0]

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().batch(1)
train_data_iter = iter(train_data)

LinearRegression_model = LinearRegression()

In [31]:
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)

  batch_xs = tf.expand_dims(batch_xs, 0)
  train_step(LinearRegression_model, batch_xs, batch_ys)

In [32]:
x_test = [3.5, 5.0, 5.5, 6.0]
test_data = tf.data.Dataset.from_tensor_slices((x_test))
test_data = test_data.batch(1)

In [33]:
for batch_x_test in test_data:
  batch_x_test = tf.expand_dims(batch_x_test, 0)

  print(tf.squeeze(LinearRegression_model(batch_x_test), 0).numpy())

[-5.859068]
[-8.370096]
[-9.207107]
[-10.044116]
